<a href="https://colab.research.google.com/github/leong-chee-ray-darius/Computely/blob/main/Courseworkhoho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit pyngrok -q
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.0/331.0 kB 27.2 MB/s eta 0:00:00


In [ ]:
#@title Make Directory structure
!mkdir -p data helper_functions logics .streamlit

In [ ]:
#import cp+ textbook pdf from gdrive
from google.colab import drive
drive.mount('/content/gdrive')


KeyboardInterrupt: 

In [ ]:
%%writefile readwritepdf.py
#Read pdf
from pypdf import PdfReader
import json
pdf_path="/content/gdrive/My Drive/Computing/G3 Computing Textbook V4 (Online) (2025-06).pdf"
output_json_path = "/content/gdrive/My Drive/Computing/textbook_data.json"
reader = PdfReader(pdf_path)
pdf_data = []
print("Extracting text...")
for i, page in enumerate(reader.pages):
    page_text = page.extract_text()
    pdf_data.append({
        "page_number": i + 1,
        "content": page_text.strip()
    })
with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(pdf_data, f, ensure_ascii=False, indent=4)
print(f"File successfully saved to: {output_json_path}")

Writing readwritepdf.py


In [ ]:
# @title Setup the secrets file
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY') # Remember to fill this up in the Secrets tab in Google Colab

with open('.streamlit/secrets.toml', 'w') as outfile:
    outfile.write(f"""
OPENAI_API_KEY="{OPENAI_API_KEY}"
OPENAI_MODEL_NAME="gpt-4o-mini"
""")

In [ ]:
j%%writefile app.py
from openai import OpenAI
import streamlit as st
import pandas as pd
import io
import time
import json
st.set_page_config(page_title="Computing Companion", layout="wide")
JSON_PATH = "/content/gdrive/My Drive/Computing/textbook_data.json"
def load_textbook_data():
    try:
        with open(JSON_PATH, 'r', encoding='utf-8') as f:
            return json.load(f)
    except:
        return []

raw_data = load_textbook_data()

STATIONERY_DATA = {
    "Computer Architecture": {
        "summary": "Covers the CPU, RAM, secondary storage, and data/address buses.",
        "keywords": ["processor", "CPU", "RAM", "memory", "storage", "bus", "USB", "HDMI"],
        "quiz": [{"q": "What is the function of the address bus?", "a": "Transports memory locations from the processor to memory."}]
    },
    "Data Representation": {
        "summary": "Number systems (Binary, Denary, Hexadecimal) and text representation.",
        "keywords": ["binary", "denary", "hexadecimal", "ASCII", "bit", "byte", "negative numbers"],
        "quiz": [{"q": "How many bits are in a byte?", "a": "8"}]
    },
    "Logic Gates": {
        "summary": "Boolean logic, truth tables, and logic circuits.",
        "keywords": ["AND", "OR", "NOT", "XOR", "truth table", "Boolean", "logic circuit"],
        "quiz": [{"q": "Which gate outputs 1 only if both inputs are 1?", "a": "AND"}]
    },
    "Programming": {
        "summary": "Python basics, data types, control flow, and functions.",
        "keywords": ["python", "variable", "list", "dictionary", "loop", "function", "if statement"],
        "quiz": [{"q": "Which Python statement is used to handle files safely?", "a": "with statement"}]
    },
    "Input Validation": {
        "summary": "Techniques to ensure data entered by users is sensible and secure.",
        "keywords": ["validation", "invalid input", "check", "range check", "type check"],
        "quiz": [{"q": "Why is input validation needed?", "a": "To prevent program errors from invalid data."}]
    },
    "Testing and Debugging": {
        "summary": "Identifying bugs, error types, and designing test cases.",
        "keywords": ["bug", "syntax error", "logic error", "test case", "trace table", "debugging"],
        "quiz": [{"q": "What is a logic error?", "a": "The program runs but produces the wrong result."}]
    },
    "Algorithm Design": {
        "summary": "Decomposition, generalisation, and solving common problems.",
        "keywords": ["decomposition", "generalisation", "algorithm", "flowchart", "pseudocode"],
        "quiz": [{"q": "What is decomposition?", "a": "Breaking a complex problem into smaller parts."}]
    },
    "Software Engineering": {
        "summary": "Development stages and alternative methodologies.",
        "keywords": ["requirement gathering", "design", "development", "alpha testing", "beta testing"],
        "quiz": [{"q": "What is beta testing?", "a": "Testing done externally in the user's environment."}]
    },
    "Spreadsheets": {
        "summary": "Using cell references, formulas, and complex functions.",
        "keywords": ["formula", "relative reference", "absolute reference", "VLOOKUP", "IF", "COUNTIF"],
        "quiz": [{"q": "What does an absolute reference look like?", "a": "e.g., $A$1"}]
    },
    "Networking": {
        "summary": "Types of networks, protocols, and the Internet.",
        "keywords": ["LAN", "WAN", "protocol", "TCP/IP", "HTTP", "DNS", "packet switching"],
        "quiz": [{"q": "What is a protocol?", "a": "A set of rules for data communication."}]
    },
    "Security and Privacy": {
        "summary": "Threats (malware, phishing) and defenses (encryption, firewalls).",
        "keywords": ["privacy", "security", "malware", "phishing", "encryption", "firewall", "biometrics"],
        "quiz": [{"q": "What is phishing?", "a": "Attempting to acquire sensitive info by masquerading as a trustworthy entity."}]
    },
    "Intellectual Property": {
        "summary": "Copyright, software licenses, and piracy.",
        "keywords": ["copyright", "license", "piracy", "infringement", "open source", "proprietary"],
        "quiz": [{"q": "What is software piracy?", "a": "The unauthorized copying or distribution of software."}]
    },
    "Impact of Computing": {
        "summary": "How computing affects industries and the spread of falsehoods.",
        "keywords": ["automation", "falsehoods", "ethics", "social media", "industry impact"],
        "quiz": [{"q": "How can falsehoods spread online?", "a": "Through social media algorithms and bot accounts."}]
    },
    "Emerging Technologies": {
        "summary": "Artificial Intelligence, machine learning, and new innovations.",
        "keywords": ["AI", "machine learning", "training data", "bias", "cloud computing", "blockchain"],
        "quiz": [{"q": "What is training data?", "a": "Data used to 'teach' an AI model how to make predictions."}]
    }
}

# --- UI LAYOUT ---
st.title("Computing Study Companion")
topic = st.sidebar.selectbox("Select a Chapter:", list(STATIONERY_DATA.keys()))
mode = st.sidebar.radio("Activity:", ["Review", "Active Recall", "AI bot"])

# --- TOKEN FIX: Filter content by keyword ---
def get_filtered_context(selected_topic):
    keywords = STATIONERY_DATA[selected_topic].get("keywords", [])
    matches = []
    for page in raw_data:
        # Only grab pages that mention keywords to save tokens
        if any(key.lower() in page['content'].lower() for key in keywords):
            matches.append(page['content'])

    # Join only the first 10 matching pages (roughly 5,000 - 10,000 tokens max)
    return "\n\n".join(matches[:10])
tb_content = get_filtered_context(topic)

# --- MODES ---

if mode == "Review":
    st.header(f"Study Notes: {topic}")
    st.info(STATIONERY_DATA[topic]["summary"])
    st.button("View Textbook Page")

elif mode == "Active Recall":
    st.header("Flashcards")
    for i, item in enumerate(STATIONERY_DATA[topic]["quiz"]):
        with st.expander(f"Question {i+1}: {item['q']}"):
            st.success(f"Answer: {item['a']}")
elif mode == "AI bot":
    st.title("Ai bot")
    system_message_content= f"""

    You are a robotic, highly efficient GCE Computing assistant.
    Your goal is to provide precise, technical answers based on the provided textbook context.

    TEXTBOOK CONTEXT:
    {tb_content}

    RULES:
    1. Use the provided context to answer the user's question.
    2. Maintain a robotic, neutral tone. No emotions, no fluff.
    3. Keep answers concise and structured (use bullet points for processes).
    5. Primary Source: Use the provided textbook context.
    6. Secondary Source: If the topic is clearly about Computing (e.g., Privacy, Hardware, Internet) but not in the context, use your general knowledge to answer.
    7. Decline: Only decline if the user asks about non-computing topics (e.g., "How do I bake a cake?").
    8. Use the exact keywords from the context
    9. There are 14 topics

    EXAMPLES:

    <Examples>
    User: What is a bit?
    Assistant: A bit is the smallest unit of data in a computer, representing a binary value of 0 or 1.
    </Examples>
    """
    client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

    if "openai_model" not in st.session_state:
      st.session_state["openai_model"] = "gpt-4o-mini"

    if "messages" not in st.session_state:
      st.session_state.messages = []
      st.session_state.messages.append({"role": "system", "content": system_message_content})

    for message in st.session_state.messages:
      if message["role"] in ["user", "assistant"]:
          with st.chat_message(message["role"]):
              st.markdown(message["content"])

    if prompt := st.chat_input("What is up?"):
      st.session_state.messages.append({"role": "user", "content": prompt })
      with st.chat_message("user"):
          st.markdown(prompt)

      with st.chat_message("assistant"):
          stream = client.chat.completions.create(
              model="gpt-4o-mini",
              messages=[
                  {"role": m["role"], "content": m["content"]}
                  for m in st.session_state.messages
              ],
              temperature=0,
              stream=True,
          )
          response = st.write_stream(stream)
      st.session_state.messages.append({"role": "assistant", "content": response})


Overwriting app.py


In [ ]:
from pyngrok import ngrok
from google.colab import userdata
import os

# Terminate any existing ngrok tunnels
ngrok.kill()

# Retrieve ngrok token
ngrok_token = userdata.get('ngrok_token')
ngrok.set_auth_token(ngrok_token)

# Start ngrok tunnel
public_url = ngrok.connect(8501, proto='http')
print(f"Public URL: {public_url}")

# Run Streamlit in the background
!streamlit run app.py &

Public URL: NgrokTunnel: "https://unsensualized-susan-balkily.ngrok-free.dev" -> "http://localhost:8501"





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.139.192.37:8501

